In [87]:
## 读取数据
import pandas as pd
import numpy as np
import os 

def prepare_Data(path_name):
    csv_path = os.path.join(path_name, 'answer.csv')
    # df_raw = pd.read_csv(r'C:\Users\alanc\Desktop\answer\commit_75.68885\commit_75.68885.csv')
    df_raw = pd.read_csv(csv_path)
    # df_raw[['计划日期', '车型', '天窗', '外色描述', '车辆等级描述', '电池特征']]

    ## 格式化输入数据
    COLUMNS = {'date': '计划日期', 'kind': '车型', 'roof': '天窗', 'color': '外色描述', 'power': '电池特征', 'grade': '车辆等级描述'}
    VALUES = {}
    columns = []

    for c, name in COLUMNS.items():
        values = list(df_raw[name].unique())
        VALUES[c] = values

        s = df_raw[name].map({v: i for i, v in enumerate(values)})
        s.name = c
        columns.append(s)

    # 大颜色
    MAJOR = frozenset(
        i for i, c in enumerate(VALUES['color'])
        if c in ["白云蓝","极地白","极地白-Y","幻影银","幻影银(出租车)","极速银","极速银-Y","极速银(出租车)","夜影黑","夜影黑-Y","自由灰","自由灰-Y","素雅灰","素雅灰-Y","天青色","天青色-Y","珍珠白","全息银"])
    # 小颜色
    MINOR = frozenset(
        i for i, c in enumerate(VALUES['color'])
        if c in ["量子红","量子红-Y","冰玫粉","冰玫粉-Y","蒂芙尼蓝","星漫绿","星漫绿-Y","琉璃红","夜荧黄","黄绿荧","薄荷贝绿","烟雨青","幻光紫","广交红","闪电橙","脉冲蓝","天际灰","火焰橙","幻光紫","幻光紫-Y","琉璃红","松花黄","松花黄-Y"])
    # 双色
    BICOLOR = frozenset(
        i for i, c in enumerate(VALUES['color'])
        if '/' in c)
    # 石墨
    GRAPHITE = frozenset(
        i for i, c in enumerate(VALUES['power'])
        if '石墨' in c)
    # 四驱
    AWD = frozenset(
        i for i, c in enumerate(VALUES['power'])
        if '/' in c)
    # K3
    K3 = frozenset(
        i for i, c in enumerate(VALUES['kind'])
        if c in ['K3'])

    df_converted = pd.concat(columns,axis=1)
    df_converted['awd'] = df_converted['power'].isin(AWD)
    df_converted['major'] = np.where(df_converted['color'].isin(MAJOR), df_converted['color'], -1)
    df_converted['minor'] = np.where(df_converted['color'].isin(MINOR), df_converted['color'], -1)
    df_converted['bicolor'] = np.where(df_converted['color'].isin(BICOLOR), df_converted['color'], -1)
    df_converted['graphite'] = np.where(df_converted['power'].isin(GRAPHITE), df_converted['power'], -1)
    df_converted['k3'] = np.where(df_converted['kind'].isin(K3), df_converted['kind'], -1)

    return df_converted


## 评分

# 切换次数
def switch_count(a, b=None):
    d = np.diff(a)
    if b is not None:
        d = np.where(np.diff(b)==0, d, 0)
    return np.count_nonzero(d)

def _eval_score(df):
    df = df[['kind','roof','color','power','grade','awd','major','minor','bicolor','graphite','k3']].to_numpy()
    kind,roof,color,power,grade,awd,major,minor,bicolor,graphite,k3 = [df[:,i] for i in range(11)]
    s = [
        switch_count(c1, c2)
        for c1, c2 in [
            (kind,None),  # 车型 - 切换
            (roof,None),  # 天窗 - 切换
            (color,None), # 颜色 - 切换
            (power,None), # 电池 - 切换
            (grade,None), # 配置等级 - 切换
            (awd, kind),  # 四驱 - 单批内集中
        ]
    ] + [
        0 # K3 - 单批内集中 (不知所云)
    ]

    gaps = []
    counts = []
    evens = []

    for c, glb, clb, cub, e in [
        (minor,    60,   15,   30,   False),   # 小颜色 - 批与批间隔, 单批内数量
        (bicolor,  60,   None, 4,    True),    # 双色车 - 批与批间隔, 单批内数量, 均匀性
        (major,    None, 15,   None, False),   # 大颜色 - 单批内数量
        (graphite, 30,   None, 1,    True),    # 石墨电池 - 批与批间隔, 单批内数量, 均匀性
        (k3,       None, None, None, True),    # K3 - 均匀性
        ]:
        # 批的起始位置
        p, = np.where(np.diff(c, prepend=-1,append=-1))
        # 批是否为小颜色，除掉开头末尾非小颜色的
        b = c[p[:-1]] >= 0
        # 小颜色批数
        total = np.count_nonzero(b)

        have_gap = glb is not None
        if total < 2:
            if have_gap:
                gaps.append(0)
            if e:
                gv = 0.0
        elif e or have_gap:
            # 小颜色批的间隔 (小颜色批的起始位置 - 上一个小颜色批的结束位置)
            g = np.extract(b, p)[1:] - np.extract(b, p[1:])[:-1]
            if have_gap:
                gaps.append(np.count_nonzero(g < glb)/len(g))
            if e:
                gv = np.std(g)/np.mean(g)

        have_count = (clb is not None) or (cub is not None)
        if total < 1:
            if have_count:
                counts.append(0)
            nv = 0.0
        elif e or have_count:
            # 小颜色各批的数量
            n = np.extract(b, np.diff(p))
            if have_count:
                count = (
                    (0 if clb is None else np.count_nonzero(n < clb)) +
                    (0 if cub is None else np.count_nonzero(n > cub)))
                counts.append(count / total)
            if e:
                nv = np.std(n)/np.mean(n)
                

        if e:
            evens.append(nv+gv)

    s.extend(gaps)
    s.extend(counts)
    s.extend(evens)
    return np.array(s)

def eval_score(df, indices):
    df_all = df.take(indices)
    weight_coefficient = np.array([50, 4, 2, 0.5, 0.5, 1, 1, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1])
    # weight_coefficient = np.array([10, 4, 2, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])
    df = pd.DataFrame(np.vstack([_eval_score(df_all[df_all['date'] == i])*weight_coefficient for i in range(53)]))
    df.columns = [i+1 for i in range(17)]
    final_result = df.sum()
    return final_result




In [88]:
import os

def get_subfolder_paths(folder_path):
    subfolder_paths = []
    subfolder_names = []
    for entry in os.scandir(folder_path):
        if entry.is_dir():
            subfolder_paths.append(os.path.join(folder_path, entry.name))
            subfolder_names.append(entry.name)
    return subfolder_paths, subfolder_names

# 指定文件夹路径
folder_path = r"C:\Users\alanc\Desktop\answer"

# 获取所有子文件夹的完整路径
subfolder_paths,subfolder_names = get_subfolder_paths(folder_path)
LOSS = []

for _, path in enumerate(subfolder_paths):
    df_converted = prepare_Data(path)
    LOSS.append(sum(eval_score(df_converted, list(range(len(df_converted))))))

# 转换为DataFrame
df = pd.DataFrame({'Score': subfolder_names, 'My_Loss': LOSS})
df

C:\Users\alanc\AppData\Local\Temp\ipykernel_24064\2920719004.py:117: RuntimeWarning: invalid value encountered in scalar divide
  gv = np.std(g)/np.mean(g)
C:\Users\alanc\AppData\Local\Temp\ipykernel_24064\2920719004.py:117: RuntimeWarning: invalid value encountered in scalar divide
  gv = np.std(g)/np.mean(g)
C:\Users\alanc\AppData\Local\Temp\ipykernel_24064\2920719004.py:117: RuntimeWarning: invalid value encountered in scalar divide
  gv = np.std(g)/np.mean(g)
C:\Users\alanc\AppData\Local\Temp\ipykernel_24064\2920719004.py:117: RuntimeWarning: invalid value encountered in scalar divide
  gv = np.std(g)/np.mean(g)
C:\Users\alanc\AppData\Local\Temp\ipykernel_24064\2920719004.py:117: RuntimeWarning: invalid value encountered in scalar divide
  gv = np.std(g)/np.mean(g)
C:\Users\alanc\AppData\Local\Temp\ipykernel_24064\2920719004.py:117: RuntimeWarning: invalid value encountered in scalar divide
  gv = np.std(g)/np.mean(g)
C:\Users\alanc\AppData\Local\Temp\ipykernel_24064\2920719004.py:

,Score,My_Loss
0,-102.72061,43522.172556
1,-21.77493,31495.805478
2,1.80054,17218.142000
3,2.51641,27453.416660
4,75.68885,16149.569319
5,77.42668,16132.852744
6,77.59533,16120.357983
7,answer,57329.987274
8,answer (1),30295.607375
9,answer (2),35456.365644
